<a href="https://colab.research.google.com/github/Amit198181/JBVNL/blob/main/110_Tower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import files
import os

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving TOWER_APR_AUG.xlsx to TOWER_APR_AUG.xlsx
User uploaded file "TOWER_APR_AUG.xlsx" with length 2199278 bytes


# Task
Analyze the data in the file "Distribution_Losses.xlsx" to determine if the contract demand for any consumer should be enhanced based on the provided criteria. Display the first 5 rows of the relevant data.

## Load the data

### Subtask:
Read the Excel file into a pandas DataFrame.


**Reasoning**:
Import the pandas library and read the Excel file into a DataFrame. Then display the first 5 rows.



In [2]:
import pandas as pd

df = pd.read_excel("TOWER_APR_AUG.xlsx")
display(df.head())

,ZONE,CIRCLE,DIVISION,CONSNO,ACCOUNT_NO,CONSUMER_STATUS,CONSUMER_NAME,TARIFF_CODE,CONTRACTED_LOAD,CONSUMER_TYPE,...,LOAD_FACTOR_HTS_202506,SUPPLY_HOURS_HTS_202506,BILLED_UNITS_HTS_202507,MDI_HTS_202507,LOAD_FACTOR_HTS_202507,SUPPLY_HOURS_HTS_202507,BILLED_UNITS_HTS_202508,MDI_HTS_202508,LOAD_FACTOR_HTS_202508,SUPPLY_HOURS_HTS_202508
0,NaN,NaN,NaN,NaN,11220368906,NaN,NaN,NaN,NaN,NaN,...,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RANCHI,RANCHI,RANCHI WEST,B33719,11220428916,LIVE,INDUS TOWERS LIMITED,CS Urban,7.0,TOWER INDUS,...,NaN,NaN,NaN,NaN,NaN,NaN,7113.0,5.0,0.0,0
2,RANCHI,RANCHI,RANCHI WEST,MD21603,11220434050,LIVE,BHARTI INFRATEL LTD,CS Urban,12.0,TOWER INDUS,...,NaN,NaN,NaN,NaN,NaN,NaN,974.0,8.0,0.0,0
3,NaN,NaN,NaN,NaN,11220439587,NaN,NaN,NaN,NaN,NaN,...,0.0,0,1435.0,8.02,0.0,0,NaN,NaN,NaN,NaN
4,RANCHI,RANCHI,RANCHI WEST,KH4002,11220519236,LIVE,INTEX WELL (PHED) SOFINADI,CS Urban,80.0,TOWER ATC,...,0.0,0,1160.0,65.00,0.0,0,1115.0,71.0,0.0,0


## Calculate maximum demand

### Subtask:
Identify and calculate the maximum demand from the relevant column in the DataFrame.


**Reasoning**:
Identify the columns containing "MDI_HTS" and calculate the maximum value across these columns for each row to create the 'MAX_DEMAND' column.



In [3]:
mdi_columns = [col for col in df.columns if "MDI_HTS" in col]
df['MAX_DEMAND'] = df[mdi_columns].max(axis=1)
display(df[['ACCOUNT_NO', 'MAX_DEMAND']].head())

,ACCOUNT_NO,MAX_DEMAND
0,11220368906,6.00
1,11220428916,5.00
2,11220434050,8.00
3,11220439587,8.02
4,11220519236,71.00


## Apply the condition

### Subtask:
Check if the maximum demand exceeds the contract demand by 10% or more at least four times.


**Reasoning**:
Identify the columns containing monthly MDI values, count how many times the monthly MDI exceeds 110% of the contracted load for each consumer, and store the count in a new column.



In [4]:
mdi_columns = [col for col in df.columns if "MDI_HTS" in col]
df['EXCEED_COUNT'] = (df[mdi_columns].gt(df['CONTRACTED_LOAD'] * 1.1, axis=0)).sum(axis=1)
display(df[['ACCOUNT_NO', 'CONTRACTED_LOAD', 'MAX_DEMAND', 'EXCEED_COUNT']].head())

,ACCOUNT_NO,CONTRACTED_LOAD,MAX_DEMAND,EXCEED_COUNT
0,11220368906,NaN,6.00,0
1,11220428916,7.0,5.00,0
2,11220434050,12.0,8.00,0
3,11220439587,NaN,8.02,0
4,11220519236,80.0,71.00,0


## Determine enhanced contract demand

### Subtask:
If the condition is met, calculate the average of the four highest maximum demand recordings.


**Reasoning**:
Filter the DataFrame for rows where 'EXCEED_COUNT' is 4 or more, select the monthly MDI columns, sort them in descending order, select the top 4, calculate the mean, and assign it to a new column 'ENHANCED_CONTRACT_DEMAND'. For rows where 'EXCEED_COUNT' is less than 4, set 'ENHANCED_CONTRACT_DEMAND' to NaN. Finally, display the first 5 rows including the new column.



In [5]:
mdi_columns = [col for col in df.columns if "MDI_HTS" in col]
df_filtered = df[df['EXCEED_COUNT'] >= 4].copy()
df_filtered['ENHANCED_CONTRACT_DEMAND'] = df_filtered[mdi_columns].apply(lambda row: sorted(row, reverse=True)[:4], axis=1).apply(lambda x: pd.Series(x).mean())
df['ENHANCED_CONTRACT_DEMAND'] = df_filtered['ENHANCED_CONTRACT_DEMAND']
display(df[['ACCOUNT_NO', 'EXCEED_COUNT', 'ENHANCED_CONTRACT_DEMAND']].head())

,ACCOUNT_NO,EXCEED_COUNT,ENHANCED_CONTRACT_DEMAND
0,11220368906,0,NaN
1,11220428916,0,NaN
2,11220434050,0,NaN
3,11220439587,0,NaN
4,11220519236,0,NaN


## Display results

### Subtask:
Display the relevant data, including the original and potentially enhanced contract demand, up to 5 rows as requested.


**Reasoning**:
Display the first 5 rows of the selected columns as requested in the instructions.



In [6]:
relevant_columns = ['ACCOUNT_NO', 'CONTRACTED_LOAD', 'MAX_DEMAND', 'EXCEED_COUNT', 'ENHANCED_CONTRACT_DEMAND']
display(df[relevant_columns].head())

,ACCOUNT_NO,CONTRACTED_LOAD,MAX_DEMAND,EXCEED_COUNT,ENHANCED_CONTRACT_DEMAND
0,11220368906,NaN,6.00,0,NaN
1,11220428916,7.0,5.00,0,NaN
2,11220434050,12.0,8.00,0,NaN
3,11220439587,NaN,8.02,0,NaN
4,11220519236,80.0,71.00,0,NaN


## Summary:

### Data Analysis Key Findings

*   The maximum demand for each consumer was calculated by taking the maximum value across the monthly MDI columns.
*   The number of times each consumer's monthly maximum demand exceeded 110% of their contracted load was determined and stored in the 'EXCEED\_COUNT' column.
*   For consumers whose maximum demand exceeded 110% of their contracted load at least four times, the average of their four highest monthly maximum demand recordings was calculated as the 'ENHANCED\_CONTRACT\_DEMAND'.
*   The final output displays the 'ACCOUNT\_NO', 'CONTRACTED\_LOAD', 'MAX\_DEMAND', 'EXCEED\_COUNT', and 'ENHANCED\_CONTRACT\_DEMAND' for the first 5 rows of the dataset.

### Insights or Next Steps

*   Identify the accounts where the 'ENHANCED\_CONTRACT\_DEMAND' is significantly higher than the 'CONTRACTED\_LOAD' to prioritize potential contract adjustments.
*   Investigate the reasons for the high demand exceedances for the identified accounts, such as changes in operational patterns or equipment usage.


In [7]:
df.to_excel("enhanced_contract_demand.xlsx", index=False)
files.download("enhanced_contract_demand.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>